In [194]:
# This notebook reads in the discretised input data and all timesteps that are longer than 4 hours 
# are split into multiple rows, each corresponding to 4 hour interval

In [4]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pandas import DataFrame

In [6]:
disc_inp_data = pd.read_csv("../data/discretised_input_data.csv")
disc_inp_data.head()

,bloc,icustayid,charttime,gender,age,elixhauser,re_admission,died_in_hosp,mortality_90d,Weight_kg,...,SIRS,median_dose_vaso,max_dose_vaso,input_total_tev,input_4hourly_tev,output_total,output_4hourly,cumulated_balance_tev,vaso_input,iv_input
0,1,3,7245052800,0,17639.826435,0.0,0,0,1,77.5,...,4,0.0,0.0,3500.0,2100.0,230.0,230.0,3270.0,0.0,4.0
1,2,3,7245067200,0,17639.826435,0.0,0,0,1,77.5,...,3,0.0,0.0,5499.0,1999.0,697.0,467.0,4802.0,0.0,4.0
2,3,3,7245081600,0,17639.826435,0.0,0,0,1,77.5,...,3,0.0,0.0,5604.0,105.0,2302.0,1605.0,3302.0,0.0,2.0
3,4,3,7245096000,0,17639.826435,0.0,0,0,1,77.5,...,3,0.0,0.0,5664.0,60.0,2922.0,620.0,2742.0,0.0,2.0
4,5,3,7245110400,0,17639.826435,0.0,0,0,1,77.5,...,3,0.0,0.0,5724.0,60.0,3352.0,430.0,2372.0,0.0,2.0


In [7]:
disc_inp_data['SIRS'].value_counts()

1    78594
2    74063
3    41299
0    39953
4     8541
Name: SIRS, dtype: int64

In [199]:
srs = []
for i in disc_inp_data.index:
    repeat = disc_inp_data.loc[i, 'dif']
    if repeat > 0:
        for j in range(int(repeat)):
            new_row = disc_inp_data.loc[i]
            new_row.loc["charttime"] = disc_inp_data.loc[i, 'charttime'] + 14400*(j+1)
            srs.append(list(new_row))



/Users/haripriyamehta/miniconda3/envs/secure_compute/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [200]:
df = pd.DataFrame(srs, columns=disc_inp_data.columns)

df['icustayid'] = df['icustayid'].astype(int)
df['bloc'] = df['bloc'].astype(int)
df['charttime'] = df['charttime'].astype(int)
df['gender'] = df['gender'].astype(int)
df['re_admission'] = df['re_admission'].astype(int)
df['died_in_hosp'] = df['died_in_hosp'].astype(int)
df['mortality_90d'] = df['mortality_90d'].astype(int)
df['mechvent'] = df['mechvent'].astype(int)
df['SOFA'] = df['SOFA'].astype(int)
df['SIRS'] = df['SIRS'].astype(int)

In [203]:
new_disc_inp_data = pd.concat([disc_inp_data, df])
new_disc_inp_data = new_disc_inp_data.sort_values(by=['icustayid', 'charttime'])      
new_disc_inp_data = new_disc_inp_data.reset_index(drop=True)
new_disc_inp_data = new_disc_inp_data.drop(columns = ["dif"])
new_disc_inp_data.head()

In [206]:
new_disc_inp_data.to_csv("../data/full_discretised_input_data.csv", index=False)